In [ ]:
# MAT 201A Spring 2017
# HW 2
# karl yerkes
# karl.yerkes@gmail.com
# Make Nick Cave into music...

In [ ]:
%pylab inline
rcParams['figure.figsize'] = (20, 4) #wide graphs by default
from ipywidgets import widgets, interact
from IPython.display import Audio
from scipy.io import wavfile

## Read in an image

In [ ]:
i = imread('media/nickcave.png')
imshow(i)
pass

## Convert to greyscale

In [ ]:
R = i[:, :, 0]
G = i[:, :, 1]
B = i[:, :, 2]
grey = R * 299. / 1000 + G * 587. / 1000 + B * 114. / 1000
grey.shape

In [ ]:
print("min:%f mean:%f median:%f max:%f" % (grey[:,0].min(), grey[:,0].mean(), median(grey[:,0]), grey[:,0].max()))

## Set DC to 0
We "stack" a row of zeros onto the bottom of the image which represents the first fft bin, the DC bin. With this image of Nick Cave, this stacking has the effect of making the height 1024, a power of 2.

In [ ]:
grey = vstack((grey, zeros_like(grey[-1:])))
shape(grey)

## Give it [Hermitian](https://en.wikipedia.org/wiki/Hermitian_function) symmetry

In [ ]:
mystft_ = vstack((flip(grey, 0), grey))
imshow(mystft_)
mystft_.shape

## Scale by the size of the FFT

In [ ]:
mystft_ *= mystft_.shape[0] / 2

## Randomize phases

In [ ]:
mystft = mystft_ * exp(-1j * uniform(-pi, pi, shape(mystft_)))

## ...or don't

In [ ]:
#mystft = mystft_ * exp(-1j * zeros_like(mystft_))

## Take the inverse FFT

In [ ]:
imystft = fft.ifft(mystft, axis = 0)
# the axis = 0 means to treat columns as FFT data

In [ ]:
# here's an alternative that takes the iFFT on each loop
#sound = zeros(2048 * 1433 + 10000) * 1j
#for column in range(1433):
#    start = column * 2048
#    sound[start:start+2048] = fft.ifft(myfft[:,column])

## Build the sound

In [ ]:
# allocate a lot of 0s
data = zeros(2048 * 1433 + 10000) * 1j
for column in range(1433):
    # copy in each piece
    start = column * 2048
    data[start:start+2048] = imystft[:,column]
data = real(data)
sound = data / amax(abs(data))
sound *= 0.5

In [ ]:
Audio(data = sound, rate = 44100)

In [ ]:
wavfile.write('result/nickcave.wav', 44100, (sound * 2**15).astype(int16))

In [ ]:
sampleRate, sample = wavfile.read('result/nickcave.wav')
specgram(sample, Fs = 44100, NFFT=1433)
pass

## (Optional) Use windows and overlapping

In [ ]:
window = hamming(2048)
#window = bartlett(2048)
plot(window)
window.shape

## Build the sound

In [ ]:
data = zeros(int(2048 * 1433 / 2 + 4000)) * 1j
data[:2048] += window *  fft.ifft(imystft[:,0])
for column in range(1, 1433):
    start = column * 1024
    data[start:start+2048] += window * imystft[:,column]
data = real(data)
sound = data / amax(abs(data))
sound *= 0.5

In [ ]:
plot(sound)
pass

In [ ]:
Audio(data = sound, rate = 44100)

In [ ]:
wavfile.write('result/nickcave_windowed.wav', 44100, (sound * 2**15).astype(int16))

In [ ]:
sampleRate, sample = wavfile.read('result/nickcave_windowed.wav')
specgram(sample, Fs = 44100, NFFT=1433)
pass

Now go watch [this video](https://www.youtube.com/watch?v=M9xMuPWAZW8&feature=youtu.be&t=300) of the spectrogram of Aphex Twin's track named...

$$\Delta M_i^{-1} = -\alpha \sum_{n=1}^N D_i[n] \left[ \sum_{j \in C[i]} F_{ji}[n-1]+Fext_i[n^{-1}]\right]$$